In [1]:
%load_ext autoreload
%autoreload 2

# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import scipy as sp
import wandb

import warnings

warnings.filterwarnings("ignore")
sys.path.append("../../")
device = "cuda" if torch.cuda.is_available() else "cpu"

from lightning_modules.jetGNN.submodels.interaction_gnn import InteractionGNN
from lightning_modules.jetGNN.submodels.gravnet import GravNet

In [5]:
# Load the config file
config_file = "jet_tag_config_2.yaml"
with open(config_file, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
model = InteractionGNN(config)

In [6]:
model = GravNet(config)

In [5]:
# For quick debugging
trainset, valset, testset = model.trainset, model.valset, model.testset

In [7]:
model.trainset, model.valset, model.testset = trainset, valset, testset

NameError: name 'trainset' is not defined

## Test Dataloading

In [8]:
model.setup(stage="fit")

Loading torch files
Fri Sep 23 02:33:13 2022


100%|██████████| 1/1 [00:54<00:00, 54.26s/it]


Building events
Fri Sep 23 02:36:14 2022
Testing sample quality


100%|██████████| 90000/90000 [00:09<00:00, 9585.38it/s]


Testing sample quality


100%|██████████| 20000/20000 [00:02<00:00, 9551.58it/s]


Testing sample quality


100%|██████████| 20000/20000 [00:02<00:00, 9443.60it/s]


Defining figures of merit
Fri Sep 23 02:36:31 2022


In [22]:
sample = model.trainset[0]

In [23]:
sample

Data(x=[37, 4], y=0, log_pt=[37], log_E=[37], delta_eta=[37], delta_phi=[37], delta_pt=[37], delta_E=[37], delta_R=[37], edge_index=[2, 1332])

## Train Model

In [9]:
logger = WandbLogger(
    project=config["project"], group="InitialTest", save_dir=config["artifacts"]
)
trainer = Trainer(
    gpus=1, max_epochs=config["max_epochs"], logger=logger
)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Defining figures of merit


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: murnanedaniel. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Fri Sep 23 02:37:01 2022



  | Name            | Type       | Params
-----------------------------------------------
0 | feature_encoder | Sequential | 10.2 K
1 | grav_convs      | ModuleList | 105 K 
2 | output_network  | Sequential | 21.5 K
-----------------------------------------------
137 K     Trainable params
0         Non-trainable params
137 K     Total params
0.549     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_SUPPORTED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`